## **Deep Neural Network 만들기**

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

#### 필요한 함수 만들기

In [2]:
def sigmoid(Z):
    
    A = 1 / (1+np.exp(-Z))
    cache = Z
    
    return A, cache

def relu(Z):
    
    A = np.maximum(0, Z)
    cache = Z
    
    return A, cache

In [3]:
def sigmoid_backward(dA, cache):
    
    Z = cache
    
    s = 1 / (1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    return dZ

def relu_backward(dA, cache):
    
    Z = cache
    dZ = np.array(dA, copy=True)
    
    dZ[Z <= 0] = 0
    
    return dZ

### 파라미터 초기화
example) layer_dims = [4, 5, 3, 2, 1] ....

In [4]:
def initialize_parameters_deep(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters["W"+str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b'+str(l)] = np.zeros((layer_dims[l], 1))
    
    return parameters

#### Z와 cache(A, W, b) 값 저장

In [15]:
# Z 값 구하기
def linear_forward(A,W,b):
    Z = np.dot(W, A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

#### 활성화 함수를 거쳐 A값과 cache 저장
linear_cache = A_prev, W, b  
activaton_cache = Z

caches = (linear_cache, activaton_cache)

In [6]:
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == 'sigmoid':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activaton_cache = sigmoid(Z)     # sigmoid(Z) --> return A, cache=Z
        
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activaton_cache = relu(Z)     # relu(Z) --> return A, cache=Z
        
    assert(A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activaton_cache)
    
    return A, cache

### layer L개 forward

In [7]:
def L_model_forward(X, parameters):
    
    caches = []    # append로 계속 담기위해
    L = len(parameters) // 2
    A = X
    
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b' + str(l)], 'relu')
        caches.append(cache)   # 층이 변하면서 바뀌는 파라미터(A,W,b,Z)를 저장하기 위해
        
    AL, cache = linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)], 'sigmoid')
    caches.append(cache)     # 마지막 층의 파라미터 저장
    
    assert(AL.shape == (1, X.shape[1]))
    
    return AL, caches

### Cost함수 계산

In [8]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]
    
    cost = np.sum ( Y * np.log(AL) + (1-Y)*np.log(1-AL) ) / -m
    cost = np.squeeze(cost)
    
    assert(cost.shape == ())
    
    return cost

#### Backward

In [9]:
def linear_backward(dZ, cache):
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ, A_prev.T)   / m
    db = np.sum(dZ, axis=1, keepdims=True) / m
    dA_prev = np.dot(W.T, dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

#### 활성화함수 거친 backward

In [10]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activaton_cache = cache

    if activation == 'relu':
        dZ = relu_backward(dA, activaton_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    if activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activaton_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db



### layer L backward

In [11]:
def L_model_backward(AL, Y, caches):
    
    L = len(caches)
    m = Y.shape[1]
    grads = {}
    Y = Y.reshape(AL.shape)
    
    dAL = - Y / AL + (1-Y) / (1-AL)
    
    current_cache = caches[L-1]
    
    grads['dA'+str(L-1)], grads['dW'+str(L)], grads['db'+str(L)] = linear_activation_backward(dAL, current_cache, 'sigmoid')
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads['dA'+str(l+1)], current_cache, 'relu')
        grads['dA'+str(l)] = dA_prev_temp
        grads['dW'+str(l+1)] = dW_temp
        grads['db'+str(l+1)] = db_temp
        
    return grads

### 파라미터 Update

In [12]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2
    
    for l in range(L):
        parameters['W' + str(l+1)] = parameters['W' + str(l+1)] - learning_rate * grads['dW' + str(l+1)]
        parameters['b' + str(l+1)] = parameters['b' + str(l+1)] - learning_rate * grads['db' + str(l+1)]
        
    return parameters